In [3]:
# IMPORT LIBRARIES:
import matplotlib.pyplot as plt 
import matplotlib.image as img
import numpy as np
import cv2

In [4]:
# READING IMAGES:
in_img = cv2.imread('./ASSETS/zebra.png', cv2.IMREAD_GRAYSCALE)
a_img = np.array(in_img)
n_img = ((a_img - np.min(a_img)) * (1/(np.max(a_img) - np.min(a_img)) * 255)).astype('uint8')
convolution_input = n_img.astype(float)

In [5]:
# FUNCTIONS/HELPER FUNCTIONS:
def convolution(image, filter):
    # Convert image to float array if it's not already
    image = [[float(val) for val in row] for row in image]
    # Convert filter to float array if it's not already
    filter = [[float(val) for val in row] for row in filter]
    
    # Determine the dimensions of the image and filter
    image_height = len(image)
    image_width = len(image[0])
    filter_height = len(filter)
    filter_width = len(filter[0])
    
    # Calculate padding
    pad_height = filter_height // 2
    pad_width = filter_width // 2
    
    # Pad the image with zeros on all sides
    padded_image = [[0 for _ in range(image_width + 2 * pad_width)] for _ in range(image_height + 2 * pad_height)]
    for i in range(image_height):
        for j in range(image_width):
            padded_image[i + pad_height][j + pad_width] = image[i][j]
    
    # Prepare the img_conv array
    img_conv = [[0 for _ in range(image_width)] for _ in range(image_height)]
    
    # Apply the filter
    for y in range(image_height):
        for x in range(image_width):
            # Extract the current region of interest
            region = [[padded_image[i][j] for j in range(x, x + filter_width)] for i in range(y, y + filter_height)]
            # Perform element-wise multiplication and sum the result
            img_conv[y][x] = sum(sum(region[i][j] * filter[i][j] for j in range(filter_width)) for i in range(filter_height))
    
    return img_conv

def manual_threshold(img_in, threshold):
    
    manual_thresh_img = []
    
    for row in img_in:
        thresholded_row = []
        for pixel in row:
            thresholded_row.append(255 if pixel > threshold else 0)
        manual_thresh_img.append(thresholded_row)

    threshold_img = manual_thresh_img

    return threshold_img

def gaussian_kernel(l, sig):
    
    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    gauss = np.exp(-0.5 * ax**2 / sig**2)
    kernel = np.outer(gauss, gauss)

    return kernel / np.sum(kernel)

def imageAdjust(image_array):
    # Calculate the minimum and maximum pixel values of the image
    min_val = np.min(image_array)
    max_val = np.max(image_array)
    
    # Adjust the image array to the range [0, 255]
    adjusted_image_array = 255 * (image_array - min_val) / (max_val - min_val)
    
    # Ensure the values are within the byte range and of type uint8
    adjusted_image_array = adjusted_image_array.astype(np.uint8)
    
    return adjusted_image_array

def detect_zero_crossings(filtered_image, delta):
    filtered_image = [[float(val) for val in row] for row in filtered_image]
    
    image_height = len(filtered_image)
    image_width = len(filtered_image[0])
    
    # Initialize the edge image with zeros (all black)
    edge_image = [[0 for _ in range(image_width)] for _ in range(image_height)]
    
    # Iterate over the image (excluding the borders)
    for i in range(1, image_height - 1):
        for j in range(1, image_width - 1):
            current_pixel = filtered_image[i][j]
            neighbors = [
                filtered_image[i-1][j-1], filtered_image[i-1][j], filtered_image[i-1][j+1],
                filtered_image[i][j-1],                             filtered_image[i][j+1],
                filtered_image[i+1][j-1], filtered_image[i+1][j], filtered_image[i+1][j+1],
            ]
            
            # Check if the current pixel is positive and if any neighbor is negative
            if current_pixel > 0:
                for neighbor in neighbors:
                    if neighbor < -delta:
                        edge_image[i][j] = 255
                        break
    return edge_image

In [6]:
# A1:
# Derivative Kernels:
sigma = 2.0
filter_g = gaussian_kernel(6*int(sigma)+1, sigma)
filter_x = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
filter_y = [[1, 2, 1], [0, 0, 0], [-1, -2, -1]]



# Results:
gaussian_image = convolution(convolution_input, filter_g)
gradient_x = convolution(gaussian_image, filter_x)
gradient_y = convolution(gaussian_image, filter_y)
gradient_magnitude = np.sqrt(np.array(gradient_x)**2 + np.array(gradient_y)**2)
binary_edges = manual_threshold(gradient_magnitude, 128)




# Writing Results:
cv2.imwrite('gaussian_img.png', imageAdjust(gaussian_image))
cv2.imwrite('x_derivative.png', imageAdjust(gradient_x))
cv2.imwrite('y_derivative.png', imageAdjust(gradient_y))
cv2.imwrite('magnitude_image.png', imageAdjust(gradient_magnitude))
cv2.imwrite('binary_edges.png', imageAdjust(binary_edges))

True

In [15]:
# A2:
# Laplacian Kernel:
sigma = 1.0
filter_g = gaussian_kernel(6*int(sigma)+1, sigma)
laplacian_filter = np.array([[0,-1,0], [-1,4,-1], [-1,0,1]])

# Results:
gaussian_image = convolution(convolution_input, filter_g)
laplacian_image = convolution(gaussian_image, laplacian_filter)
delta = .0001
zero_crossings = detect_zero_crossings(np.array(laplacian_image), delta)


# Writing Results:
cv2.imwrite('laplacian_image.png', imageAdjust(laplacian_image))
cv2.imwrite('zero_crossings.png', imageAdjust(zero_crossings))

True